### misc scratch notebook

In [10]:
import json
import tensorflow as tf
import random
import string
# import tensorflow_recommenders as tfrs

from google.cloud import storage
from google.cloud import aiplatform as vertex_ai

import numpy as np
import pickle as pkl
import os
from pprint import pprint

storage_client = storage.Client(project='hybrid-vertex')

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA

### check Experiments DF

In [14]:
vertex_ai.init(
    project='hybrid-vertex',
    location='us-central1',
    # experiment=EXPERIMENT_NAME
)

In [16]:
# EXPERIMENT_NAME = "test4-full-jtv14-4thread-jtv14"

# experiment_df = vertex_ai.get_experiment_df()
# experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
# experiment_df.T

In [ ]:
file_list = os.path.join(transformed_dataset.path, '_file_list.txt')

new_lines = []
with open(file_list, 'r') as fp:
lines = fp.readlines()
new_lines.append(lines[0])
for line in lines[1:]:
  new_lines.append(line.replace('gs://', '/gcs/'))

gcs_file_list = os.path.join(transformed_dataset.path, '_gcs_file_list.txt')
with open(gcs_file_list, 'w') as fp:
fp.writelines(new_lines)

## check feature functions

In [12]:
# Generate unique ID to help w/ unique naming of certain pieces
ID = "".join(random.choices(string.ascii_lowercase + string.digits, k=3))
ID

'tyj'

In [3]:
def return_feats(max_playlist_length):
    feats = {
        # ===================================================
        # candidate track features
        # ===================================================
        "track_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),            
        "track_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "artist_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "artist_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "album_uri_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),           
        "album_name_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
        "duration_ms_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),      
        "track_pop_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),      
        "artist_pop_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "artist_genres_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "artist_followers_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        # "track_pl_titles_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "track_danceability_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_energy_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_key_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "track_loudness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_mode_can":tf.io.FixedLenFeature(dtype=tf.string, shape=()),
        "track_speechiness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_acousticness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_instrumentalness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_liveness_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_valence_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "track_tempo_can":tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        "time_signature_can": tf.io.FixedLenFeature(dtype=tf.string, shape=()), # track_time_signature_can

        # ===================================================
        # summary playlist features
        # ===================================================
        "pl_name_src" : tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
        'pl_collaborative_src' : tf.io.FixedLenFeature(dtype=tf.string, shape=()), 
        # 'num_pl_followers_src' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
        'pl_duration_ms_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        'num_pl_songs_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), # n_songs_pl_new | num_pl_songs_new
        'num_pl_artists_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()),
        'num_pl_albums_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
        # 'avg_track_pop_pl_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
        # 'avg_artist_pop_pl_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 
        # 'avg_art_followers_pl_new' : tf.io.FixedLenFeature(dtype=tf.float32, shape=()), 

        # ===================================================
        # ragged playlist features
        # ===================================================
        # bytes / string
        "track_uri_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "track_name_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "artist_uri_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "artist_name_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "album_uri_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "album_name_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "artist_genres_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        # "tracks_playlist_titles_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(MAX_PLAYLIST_LENGTH,)),
        "track_key_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "track_mode_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)),
        "time_signature_pl": tf.io.FixedLenFeature(dtype=tf.string, shape=(max_playlist_length,)), 

        # Float List
        "duration_ms_songs_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_pop_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "artist_pop_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "artists_followers_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_danceability_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_energy_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_loudness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_speechiness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_acousticness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_instrumentalness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_liveness_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_valence_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
        "track_tempo_pl": tf.io.FixedLenFeature(dtype=tf.float32, shape=(max_playlist_length,)),
    }
    
    return feats

In [7]:
def parse_tfrecord(example, max_playlist_length):
    """
    Reads a serialized example from GCS and converts to tfrecord
    """
    # example = tf.io.parse_single_example(
    example = tf.io.parse_example(
        example,
        # feats
        features=return_feats(max_playlist_length)
    )
    return example

def full_parse(data):
    # used for interleave - takes tensors and returns a tf.dataset
    data = tf.data.TFRecordDataset(data)
    return data

In [9]:
batch_size = 1024 #*16

valid_dir = 'spotify-data-regimes'
valid_dir_prefix = 'jtv14-8m/valid_v14' 

valid_files = []
for blob in storage_client.list_blobs(f'{valid_dir}', prefix=f'{valid_dir_prefix}'):
    if '.tfrecords' in blob.name:
        valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files).prefetch(
    tf.data.AUTOTUNE,
)

valid_dataset = valid_dataset.interleave(
    full_parse,
    num_parallel_calls=tf.data.AUTOTUNE,
    cycle_length=tf.data.AUTOTUNE, 
    deterministic=False,
).map(
    parse_tfrecord(max_playlist_length=15), 
    num_parallel_calls=tf.data.AUTOTUNE
).batch(
    batch_size
).prefetch(
    tf.data.AUTOTUNE,
).with_options(
    options
)

valid_dataset = valid_dataset.cache()

TypeError: parse_tfrecord() missing 1 required positional argument: 'example'

### write `.gitignore`

In [26]:
!pwd

/home/jupyter/jw-repo/spotify_mpd_two_tower


In [27]:
%%writefile .gitignore

__pycache__/

# Jupyter Notebook
.ipynb_checkpoints

# Custom
# bqml/components/
# bqml/bqml-pipeline-ff.json
[v2 JT] -02-build-model.ipynb

# Installer logs
pip-log.txt

# Distribution / packaging
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
share/python-wheels/
*.egg-info/
.installed.cfg
*.egg
MANIFEST

Overwriting .gitignore


## cp to central GCS bucket

In [28]:
!gsutil -m cp -r gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518  gs://jt-tfrs-central/a50-epoch

Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/candidates/candidate_embeddings.json [Content-Type=application/json]...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/model-dir/candidate_model/saved_model.pb...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/model-dir/candidate_model/variables/variables.data-00000-of-00001...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/model-dir/candidate_model/variables/variables.index...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/model-dir/query_model/saved_model.pb...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/model-dir/query_model/variables/variables.data-00000-of-00001...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfrs-jtv10/run-20221230-160518/model-dir/query_model/variables/variables.index...
Copying gs://jt-tfrs-output-v2/full-perf-2tower-tfr

In [6]:
import pickle as pkl
import os

## Inspect Vocabs

### Inspect Vocab dict

In [13]:
# os.system('gsutil cp gs://spotify-data-regimes/jtv1/vocabs/vocab_dict.pkl .') # TODO: parametrize

os.system('gsutil cp gs://two-tower-models/vocabs/vocab_dict.pkl .')  # TODO - paramterize

filehandler = open('vocab_dict.pkl', 'rb')
vocab_dict = pkl.load(filehandler)
filehandler.close()

Copying gs://two-tower-models/vocabs/vocab_dict.pkl...
/ [1 files][ 18.5 MiB/ 18.5 MiB]                                                
Operation completed over 1 objects/18.5 MiB.                                     


In [14]:
len(vocab_dict['track_name_pl'])

20000

In [15]:
import numpy as np

vocab = np.array([vocab_dict['track_name_pl']]).flatten()
vocab

array(['', '[UNK]', 'the', ..., 'days a', 'bop to', 'dirty version'],
      dtype='<U34')

In [1]:
# vocab_dict['artist_name_pl']

### Inspect Vocab Dict

In [18]:
# gs://spotify-data-regimes/jtv10/vocabs/jtv10_master_vocab_dict.pkl
os.system('gsutil cp gs://spotify-data-regimes/jtv10/vocabs/jtv10_master_vocab_dict.pkl .')  # TODO - paramterize

filehandler = open('jtv10_master_vocab_dict.pkl', 'rb')
jt_vocab_dict = pkl.load(filehandler)
filehandler.close()

Copying gs://spotify-data-regimes/jtv10/vocabs/jtv10_master_vocab_dict.pkl...
/ [1 files][ 13.2 MiB/ 13.2 MiB]                                                
Operation completed over 1 objects/13.2 MiB.                                     


In [19]:
len(jt_vocab_dict['track_name_pl'])

20000

In [25]:
jt_vocab_dict

{'pl_name_src': ['',
  '[UNK]',
  'country',
  'music',
  'rock',
  'chill',
  'summer',
  'party',
  'songs',
  'good',
  'jams',
  'rap',
  'playlist',
  'the',
  'new',
  'my',
  'oldies',
  'old',
  'christmas',
  'mix',
  'workout',
  '2017',
  'vibes',
  'throwback',
  '2016',
  'car',
  'classic',
  'work',
  '90s',
  'school',
  'road',
  'worship',
  '2015',
  'lit',
  'love',
  'dance',
  'feels',
  'stuff',
  'up',
  'hop',
  'pop',
  'best',
  'hip',
  'out',
  '80s',
  'it',
  'trip',
  'throwbacks',
  'wedding',
  'edm',
  'hip hop',
  'classics',
  'disney',
  'of',
  'time',
  'road trip',
  'old school',
  'classic rock',
  'fall',
  'gym',
  'tunes',
  'slow',
  'christian',
  'random',
  'jamz',
  '17',
  'spring',
  'happy',
  'alternative',
  'all',
  'for',
  'rb',
  'run',
  'feel',
  'running',
  'study',
  'indie',
  'i',
  'me',
  '2014',
  'hype',
  'driving',
  'favorites',
  'spanish',
  'feel good',
  'house',
  'day',
  '2',
  '16',
  'sleep',
  'back',
 

## Inspect train output

In [3]:
!pwd

/home/jupyter/jw-repo/spotify_mpd_two_tower


In [12]:
JOB_DICT_v1 = 'train_job_dict_1.pkl'

!gsutil cp gs://jt-tfrs-output-v2/perf-2tower-tfrs-jtv10/run-20221229-214445/train_job_dict.pkl $JOB_DICT_v1 

filehandler = open(f'{JOB_DICT_v1}', 'rb')
train_job_dict_1 = pkl.load(filehandler)
filehandler.close()

train_job_dict_1

Copying gs://jt-tfrs-output-v2/perf-2tower-tfrs-jtv10/run-20221229-214445/train_job_dict.pkl...
/ [1 files][  1.8 KiB/  1.8 KiB]                                                
Operation completed over 1 objects/1.8 KiB.                                      


{'name': 'projects/934903580331/locations/us-central1/customJobs/6623719866928988160',
 'displayName': 'train-sp-2tower-tfrs-jtv10-v6',
 'jobSpec': {'workerPoolSpecs': [{'machineSpec': {'machineType': 'a2-highgpu-1g',
     'acceleratorType': 'NVIDIA_TESLA_A100',
     'acceleratorCount': 1},
    'replicaCount': '1',
    'diskSpec': {'bootDiskType': 'pd-ssd', 'bootDiskSizeGb': 100},
    'containerSpec': {'imageUri': 'gcr.io/hybrid-vertex/sp-2tower-tfrs-jtv10-v6-training',
     'command': ['python', 'two_tower_jt/task.py'],
     'args': ['--project=hybrid-vertex',
      '--train_output_gcs_bucket=jt-tfrs-output-v2',
      '--train_dir=spotify-data-regimes',
      '--train_dir_prefix=jtv10/valid_v9',
      '--valid_dir=spotify-data-regimes',
      '--valid_dir_prefix=jtv10/valid_v9',
      '--candidate_file_dir=spotify-data-regimes',
      '--candidate_files_prefix=jtv10/candidates',
      '--experiment_name=perf-2tower-tfrs-jtv10',
      '--experiment_run=run-20221229-214445',
      '--nu

#### another train job

In [11]:
JOB_DICT_v2 = 'train_job_dict_2.pkl'

!gsutil cp gs://jt-tfrs-output-v2/pipe-dev-2tower-tfrs-jtv10/run-20221229-150824/train_job_dict.pkl $JOB_DICT_v2

filehandler = open(f'{JOB_DICT_v2}', 'rb')
train_job_dict_2 = pkl.load(filehandler)
filehandler.close()

train_job_dict_2

Copying gs://jt-tfrs-output-v2/pipe-dev-2tower-tfrs-jtv10/run-20221229-150824/train_job_dict.pkl...
/ [1 files][  1.7 KiB/  1.7 KiB]                                                
Operation completed over 1 objects/1.7 KiB.                                      


{'name': 'projects/934903580331/locations/us-central1/customJobs/3501212397161414656',
 'displayName': 'train-sp-2tower-tfrs-jtv10-v4',
 'jobSpec': {'workerPoolSpecs': [{'machineSpec': {'machineType': 'a2-highgpu-1g',
     'acceleratorType': 'NVIDIA_TESLA_A100',
     'acceleratorCount': 1},
    'replicaCount': '1',
    'diskSpec': {'bootDiskType': 'pd-ssd', 'bootDiskSizeGb': 100},
    'containerSpec': {'imageUri': 'gcr.io/hybrid-vertex/sp-2tower-tfrs-jtv10-v4-training',
     'command': ['python', 'two_tower_jt/task.py'],
     'args': ['--project=hybrid-vertex',
      '--train_output_gcs_bucket=jt-tfrs-output-v2',
      '--train_dir=spotify-data-regimes',
      '--train_dir_prefix=jtv10/valid_v9',
      '--valid_dir=spotify-data-regimes',
      '--valid_dir_prefix=jtv10/valid_v9',
      '--candidate_file_dir=spotify-data-regimes',
      '--candidate_files_prefix=jtv10/candidates',
      '--experiment_name=pipe-dev-2tower-tfrs-jtv10',
      '--experiment_run=run-20221229-150824',
      '

# Inspect embeddings file

In [34]:
PROJECT_ID='hybrid-vertex'

In [35]:
from google.cloud import storage
from google.cloud.storage.bucket import Bucket
from google.cloud.storage.blob import Blob

storage_client = storage.Client(project=PROJECT_ID) 

In [36]:
# previously created embedding output
# !gsutil cp gs://jt-tfrs-central/pipe-dev-2tower-tfrs-jtv10/run-20221228-210041/candidates/candidate_embeddings.json local_files/candidate_embs_20221228_210041.json

# jt-tfrs-central/a50-epoch/run-20221230-160518

In [37]:
embeddings_index_filename = 'candidate_embs_local_v2_20230103-213209.json'

BUCKET = 'jt-tfrs-central'
PATH_TO_INDEX_DIR = 'a50-epoch/run-20221230-160518'
INDEX_GCS_URI = f'gs://{BUCKET}/{PATH_TO_INDEX_DIR}/candidates-v6'

print(f"INDEX_GCS_URI: {INDEX_GCS_URI}")

DESTINATION_BLOB_NAME = embeddings_index_filename
SOURCE_FILE_NAME = embeddings_index_filename

print(f"DESTINATION_BLOB_NAME: {DESTINATION_BLOB_NAME}")
print(f"SOURCE_FILE_NAME: {SOURCE_FILE_NAME}")

INDEX_GCS_URI: gs://jt-tfrs-central/a50-epoch/run-20221230-160518/candidates-v6
DESTINATION_BLOB_NAME: candidate_embs_local_v2_20230103-213209.json
SOURCE_FILE_NAME: candidate_embs_local_v2_20230103-213209.json


In [38]:
blob = Blob.from_string(os.path.join(INDEX_GCS_URI, DESTINATION_BLOB_NAME))
blob.bucket._client = storage_client
blob.upload_from_filename(SOURCE_FILE_NAME)

# Model CheckPoints and Restore

In [7]:
# import os 

# checkpoint_dir = "gs://jt-tfrs-central/a50-epoch/run-20221230-160518/pipeline_root/"

# # os.listdir(checkpoint_dir)

# latest = tf.train.latest_checkpoint(checkpoint_dir)
# latest

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
checkpoint_dir = "gs://"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    
checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]

In [2]:
# import tensorflow as tf
# from tensorflow import keras
# import pickle as pkl

# os.environ["CLOUD_ML_PROJECT_ID"]='hybrid-vertex'

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

# from google.cloud import storage
# from google.cloud import aiplatform as vertex_ai

# from src.two_tower_jt import two_tower as tt

In [50]:
storage_client = storage.Client(project='hybrid-vertex')

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.AUTO

def full_parse(data):
    # used for interleave - takes tensors and returns a tf.dataset
    data = tf.data.TFRecordDataset(data)
    return data

batch_size = 1024 #*16

In [51]:
CANDIDATE_FILE_DIR = 'spotify-data-regimes'
CANDIDATE_PREFIX = 'jtv10/candidates' 

candidate_files = []
for blob in storage_client.list_blobs(f"{CANDIDATE_FILE_DIR}", prefix=f'{CANDIDATE_PREFIX}'):
    if '.tfrecords' in blob.name:
        candidate_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

candidate_dataset = tf.data.Dataset.from_tensor_slices(candidate_files)

parsed_candidate_dataset = candidate_dataset.interleave(
    full_parse,
    cycle_length=tf.data.AUTOTUNE, 
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False
).map(
    tt.parse_candidate_tfrecord_fn, 
    num_parallel_calls=tf.data.AUTOTUNE
).with_options(
    options
)

parsed_candidate_dataset = parsed_candidate_dataset.cache() #400 MB on machine mem

2023-01-13 09:29:28.913523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 09:29:29.698616: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 718 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [52]:
filehandler = open('vocab_dict.pkl', 'rb')
vocab_dict = pkl.load(filehandler)
filehandler.close()

In [53]:
layer_sizes=[64,32]

model = tt.TheTwoTowers(layer_sizes, vocab_dict, parsed_candidate_dataset)

LR = .1
opt = tf.keras.optimizers.Adagrad(LR)
model.compile(optimizer=opt)

2023-01-13 09:29:53.266937: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.07GiB (rounded to 1151775744)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-01-13 09:29:53.266975: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-01-13 09:29:53.266985: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 10, Chunks in use: 10. 2.5KiB allocated for chunks. 2.5KiB in use in bin. 73B client-requested in use in bin.
2023-01-13 09:29:53.266992: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-01-13 09:29:53.266999: I tenso

ResourceExhaustedError: in user code:

    File "/home/jupyter/jw-repo/spotify_mpd_two_tower/src/two_tower_jt/two_tower.py", line 1260, in None  *
        lambda x: (x['track_uri_can'], self.candidate_tower(x))), ks=(1, 5, 10)),
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ResourceExhaustedError: Exception encountered when calling layer "candidate__track__model" "                 f"(type Candidate_Track_Model).
    
    in user code:
    
        File "/home/jupyter/jw-repo/spotify_mpd_two_tower/src/two_tower_jt/two_tower.py", line 1211, in call  *
            all_embs = tf.concat(
        File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
            raise e.with_traceback(filtered_tb) from None
        File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 2105, in random_uniform
          

In [ ]:
model

In [45]:
chkpt_dir = 'gs://jt-tfrs-central/new-50e-full-jtv12/run-20230110-150417/checkpoints'

checkpoints = [chkpt_dir + "/" + name for name in os.listdir(chkpt_dir)]
checkpoints
# keras.models.load_model(chkpt_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'gs://jt-tfrs-central/new-50e-full-jtv12/run-20230110-150417/checkpoints'